# So, What is TF-IDF?
*(if you're not interested in the theory part of TF-IDF, feel free to skip to the code section.)*
![](https://monkeylearn.com/static/679ad6824cd3f362d6081c38b8ef5824/35d2d/What-is-TF-IDF-Normal.png)

### ***Okay, theory time!***

**TF-IDF** is a statistical measure that evaluates how relevant a word is to a document in a collection of documents. This is done by multiplying two metrics: how many times a word appears in a document, and the inverse document frequency of the word across a set of documents.

It has many uses, most importantly in automated text analysis, and is very useful for scoring words in machine learning algorithms for Natural Language Processing (NLP).

TF-IDF (**term frequency-inverse document frequency**) was invented for document search and information retrieval. It works by increasing proportionally to the number of times a word appears in a document, but is offset by the number of documents that contain the word. So, words that are common in every document, such as this, what, and if, rank low even though they may appear many times, since they don’t mean much to that document in particular.

However, if the word Bug appears many times in a document, while not appearing many times in others, it probably means that it’s very relevant. For example, if what we’re doing is trying to find out which topics some NPS responses belong to, the word Bug would probably end up being tied to the topic Reliability, since most responses containing that word would be about that topic.

### ***Math behind TF-IDF...***
**TF-IDF** for a word in a document is calculated by multiplying two different metrics:

- The **term frequency** of a word in a document. There are several ways of calculating this frequency, with the simplest being a raw count of instances a word appears in a document. Then, there are ways to adjust the frequency, by length of a document, or by the raw frequency of the most frequent word in a document.
- The **inverse document frequency** of the word across a set of documents. This means, how common or rare a word is in the entire document set. The closer it is to 0, the more common a word is. This metric can be calculated by taking the total number of documents, dividing it by the number of documents that contain a word, and calculating the logarithm.

So, if the word is very common and appears in many documents, this number will approach 0. Otherwise, it will approach 1.
Multiplying these two numbers results in the **TF-IDF score** of a word in a document. The higher the score, the more relevant that word is in that particular document.

To put it in more formal mathematical terms, the TF-IDF score for the word `t` in the document `d` from the document set `D` is calculated as follows:

![](https://monkeylearn.com/static/23b5e36265d19e9b42a9ae42220d257b/df264/1.png)

Where:

![](https://monkeylearn.com/static/d96cda57105351e7b75b844910ab3f73/df264/2.png)

![](https://monkeylearn.com/static/aaa7bf8149587b9b828f99e1db9f7e46/df264/3.png)

*Source:* [*https://monkeylearn.com/blog/what-is-tf-idf/*](https://monkeylearn.com/blog/what-is-tf-idf/)

**Without further ado, let's dive into the practical use-case of TF-IDF.**

**We'll be extracting the top 10 keywords from the published papers in [NeurIPS Conferences](https://www.kaggle.com/rowhitswami/nips-papers-1987-2019-updated/) from 1987 to 2019 using built-in algorithm of TD-IDF in Scikit-learn library.**

# Code

In [1]:
# Printing data files
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('../input/facebook/round_23.csv'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
# General libraries
import re, os, string
import pandas as pd

# Scikit-learn importings
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
def get_stopwords_list(stop_file_path):
    """load stop words """
    
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return list(frozenset(stop_set))

In [4]:
def clean_text(text):
    """Doc cleaning"""
    
    # Lowering text
    text = text.lower()
    
    # Removing punctuation
    text = "".join([c for c in text if c not in PUNCTUATION])
    
    # Removing whitespace and newlines
    text = re.sub('\s+',' ',text)
    
    return text

In [5]:
def sort_coo(coo_matrix):
    """Sort a dict with highest score"""
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]

    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])

    #create a tuples of feature, score
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [6]:
def get_keywords(vectorizer, feature_names, doc):
    """Return top k keywords from a doc using TF-IDF method"""

    #generate tf-idf for the given document
    tf_idf_vector = vectorizer.transform([doc])
    
    #sort the tf-idf vectors by descending order of scores
    sorted_items=sort_coo(tf_idf_vector.tocoo())

    #extract only TOP_K_KEYWORDS
    keywords=extract_topn_from_vector(feature_names,sorted_items,TOP_K_KEYWORDS)
    
    return list(keywords.keys())

In [7]:
# Constants
PUNCTUATION = """!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~""" 
TOP_K_KEYWORDS = 10 # top k number of keywords to retrieve in a ranked document
STOPWORD_PATH = "/kaggle/input/stopwords/stopwords.txt"
PAPERS_PATH = "/kaggle/input/nips-papers-1987-2019-updated/papers.csv"

# Reading data

In [8]:
data = pd.read_csv('../input/facebook/round_23.csv')
data.head()

In [9]:
data.dropna(subset=['text'], inplace=True)

# Preparing data

In [10]:
data['text'] = data['text'].apply(clean_text)

In [11]:
data.head()

In [12]:
corpora = data['text'].to_list()

# Keywords Extraction using TF-IDF

In [13]:
#load a set of stop words
stopwords=get_stopwords_list(STOPWORD_PATH)

# Initializing TF-IDF Vectorizer with stopwords
vectorizer = TfidfVectorizer(stop_words=stopwords, smooth_idf=True, use_idf=True)

# Creating vocab with our corpora
# Exlcluding first 10 docs for testing purpose
vectorizer.fit_transform(corpora[10::])

# Storing vocab
feature_names = vectorizer.get_feature_names()

# Result 


In [14]:
result = []
for doc in corpora[0:50]:
    df = {}
    df['text'] = doc
    df['top_keywords'] = get_keywords(vectorizer, feature_names, doc)
    result.append(df)
    
final = pd.DataFrame(result)
final

In [15]:
pip install cufflinks

In [16]:
from plotly.offline import init_notebook_mode,iplot
import plotly.graph_objects as go
import cufflinks as cf
# init_notebook_mode(connected=True)

In [17]:
data.head()

In [18]:
from plotly.offline import init_notebook_mode,iplot
import plotly.graph_objects as go
import cufflinks as cf
init_notebook_mode(connected=True)

In [ ]:
texto_de_publicaciones = df_shares['co']
texto_de_publicaciones = [i for i in texto_de_publicaciones if type(i) == str]